# Spam Filtering using Naïve Bayes Classifier in R

Download dataset from https://plg.uwaterloo.ca/~gvcormac/treccorpus06/ and 
check <b>Instructions</b> on the Main Program section before re-running this notebook.

<h2> Part 1: Functions Definitions </h2>

<h4> Function: setup_dataset </h4>
This function divides the dataset (trec06p) into training set (70%) and test set (30%)

In [1]:
setup_dataset <- function(inputfile){
    dataset <<- scan(file=inputfile, list(type="", doc="")) # 1st col: spam/ham; 2nd col: document filename
    dataset$doc <<- paste(data_dir, dataset$doc, sep="/");
    N <- length(dataset$doc)
    N_vec <- seq(1:length(dataset$doc))
    trainingset_idx <<- N_vec[(1:N-1)%%10<7] # using modulo, Document no. 1,2,3,4,5,6,7,...,11,12,.....
    testset_idx <<- N_vec[(1:N-1)%%10>=7] # using modulo, Document no. 8,9,10,...,18,19,20,...,28,..
    print(paste("Count of all documents: ", N))
    print(paste("Count of training set: ", length(trainingset_idx)))
    print(paste("Count of test set: ", length(testset_idx)))
}

<h4> Function: parse_document </h4>
This function reads each document from the dataset and parses each word. It also removes words from the document that will not help in the training, e.g. html tags etc.

In [2]:
parse_document <- function(document, normal){
    words <- c()
    words <- scan(file=document, "", quote = "") # scan document per word (delimited by space)
    words <- gsub("<.*>|.*=.*>|&nbsp;", "", words, perl=TRUE) # remove html tags
    words <- grep("^[\"]*[a-zA-Z]+[,.\"]*$", words, perl=TRUE, value=TRUE) # select only alpha strings, with quotes or ending with ., are ok
    words <- words[nchar(words)>2 & nchar(words)<50] # select only strings that have 3-49 letters

    words <- unique(tolower(gsub("[^[:alnum:]_]", "", words))) # remove punctuations in the words, make all lowercase and unique	
    words <- words[nchar(words)>2]
    words <- words[words!=""] # remove empty strings in the vector
    words
}

<h4> Function: construct_vocabulary </h4>
This functions constructs vocabulary.txt based on words on training set documents. It counts how many times each word appears as spam and as ham.

In [3]:
construct_vocabulary <- function(vocabularyfile, normal){
    vocabulary <- c()
    wordctr_spam <- c()
    wordctr_ham <- c()

    L <- length(trainingset_idx)
    partition <- 52
    cleanup_ctr <- 1
    lowerbound <- 1
    for(i in 1:L){ 
        if (i %% 1000 == 1 || i == L)
            message(paste("Parsing training set document #",i))
        idx <- trainingset_idx[i];
        words <- parse_document(dataset$doc[idx], normal) # parse words on each documents in the training set

        newwords <- setdiff(words,vocabulary);
        wordctr_spam[newwords] <- 0 # initialize new words to 0, so can iterate properly
        wordctr_ham[newwords] <- 0

        if(dataset$type[idx] == "spam"){ # if current document is spam (ham), iterate wordctr_spam (wordctr_ham) for all words in the document
            wordctr_spam[words] <- wordctr_spam[words]+1
        }else{
            wordctr_ham[words] <- wordctr_ham[words]+1
        }

        vocabulary <- append(vocabulary, newwords) # add new words to the current vocabulary

    }

    # this block is only for: improving classifier
    if(!normal){
        # remove least frequent words
        wordctr_total <- wordctr_spam+wordctr_ham
        wordctr_total <- wordctr_total[wordctr_total>50]
        vocabulary <- names(wordctr_total)
        wordctr_spam <- wordctr_spam[vocabulary]
        wordctr_ham <- wordctr_ham[vocabulary]

        # remove 200 most frequent words
        wordctr_total <- wordctr_spam+wordctr_ham
        wordctr_total <- sort(wordctr_total, decreasing=TRUE)
        wordctr_total <- wordctr_total[201:length(wordctr_total)]
        vocabulary <- names(wordctr_total)
        wordctr_spam <- wordctr_spam[vocabulary]
        wordctr_ham <- wordctr_ham[vocabulary]

        # take biggest differences
        wordctr_diff <- abs(wordctr_spam-wordctr_ham)
        wordctr_diff <- sort(wordctr_total, decreasing=TRUE)
        wordctr_diff <- wordctr_diff[1:200]
        vocabulary <- names(wordctr_diff)
        wordctr_spam <- wordctr_spam[vocabulary]
        wordctr_ham <- wordctr_ham[vocabulary]
    }

    vocabulary <- sort(vocabulary)
    write(paste(vocabulary, wordctr_spam[vocabulary], wordctr_ham[vocabulary], sep=","), vocabularyfile) # write to a file, space delimited
    print(paste0("New vocabulary is saved to ", vocabularyfile, "."))

    # make scope:global
    vocabulary <<- vocabulary
    wordctr_spam <<- wordctr_spam[vocabulary]
    wordctr_ham <<- wordctr_ham[vocabulary]
}

<h4> Function: load_vocabulary </h4>
This function load vocabulary.txt (for option to load existing vocabulary from file)

In [4]:
load_vocabulary <- function(vocabularyfile){
    v <- scan(file=vocabularyfile, list(w="", s=0, h=0), sep=",")
    vocabulary <<- v$w
    wordctr_spam <<- v$s
    names(wordctr_spam) <<- vocabulary # set the vocabulary words as the index for the ctr
    wordctr_ham <<- v$h
    names(wordctr_ham) <<- vocabulary # set the vocabulary words as the index for the ctr
}

<h4> Functions: compute_priors and compute_likelihoods </h4>

In [5]:
compute_priors <- function(){
    N <- length(dataset$type[trainingset_idx]) # total # of training set
    S <- table(dataset$type[trainingset_idx])["spam"] # total # of spam docs in training set
    H <- table(dataset$type[trainingset_idx])["ham"] # total # of ham docs in training set
    prior_spam <<- S/N # spam docs / all docs
    prior_ham <<- H/N # ham docs / all docs
}

compute_likelihoods <- function(index){
    lambda <- lambdalist[index]
    S <- table(dataset$type[trainingset_idx])["spam"] # total # of spam docs in training set
    H <- table(dataset$type[trainingset_idx])["ham"] # total # of ham docs in training set
    V <- length(vocabulary) # size of vocabulary
    likelihood_spam[index,] <<- (wordctr_spam+lambda)/(S+lambda*V) # spam docs where the each word appeared / all spam docs 
    likelihood_ham[index,] <<- (wordctr_ham+lambda)/(H+lambda*V) # ham docs where the each word appeared / all ham docs
}

<h4> Functions: iterate_precision_recall, compute_precision and compute_recall </h4>

In [6]:
iterate_precision_recall <- function(true_value, classification, index){
    if(true_value == "spam"){
        if(classification == "spam")
            true_pos[index] <<- true_pos[index]+1 # spam msgs classified as spam
        else
            false_neg[index] <<- false_neg[index]+1 # spam msgs misclassified as ham
    }else if(true_value == "ham"){
        if(classification == "ham")
            true_neg[index] <<- true_neg[index]+1 # ham msgs classified as ham
        else
            false_pos[index] <<- false_pos[index]+1 # ham msgs misclassified as spam
    }
}

compute_precision <- function(index){
    precision <- true_pos[index]/(true_pos[index]+false_pos[index])
    precision
}

compute_recall <- function(index){
    recall <- true_pos[index]/(true_pos[index]+false_neg[index])
    recall
}

<h4> Function: run_classifier </h4>
This function runs Naive Bayes Classifier on the test set

In [7]:
run_classifier <- function(filename, normal){
    L <- length(lambdalist)

    # initialize output files to be used, 1 per lambda value
    outputfile <- rep("", times=L)
    names(outputfile) <- lambdalist
    for(i in 1:L){
        lambda <- lambdalist[i]
        fmt_lambda <- sprintf("%.3f", lambda);
        outputfile[i] <- paste(filename, fmt_lambda, "txt", sep=".")
    }

    classification <- matrix(data="", nrow=length(lambdalist), ncol=length(testset_idx), byrow=FALSE)

    # initialize precision & recall counters
    true_pos <<- rep(0, times=L)
    false_pos <<- rep(0, times=L)
    true_neg <<- rep(0, times=L)
    false_neg <<- rep(0, times=L)
    precision <<- rep(0, times=L)
    recall <<- rep(0, times=L)

    # calculate posteriori probabilities for each documents in the test set
    for(j in 1:length(testset_idx)){
        if (j %% 500 == 1 || j == length(testset_idx))
            message(paste("Parsing test set document #",j))

        idx <- testset_idx[j]
        words <- parse_document(dataset$doc[idx], normal)
        vocab_present <- intersect(vocabulary, words) # vocab words present in document
        vocab_absent <- setdiff(vocabulary, words) # vocab words absent in document

        # calculate for each lambda
        for(i in 1:L){
            lxplog_spam <- sum(log(likelihood_spam[i,vocab_present]))+sum(log(1-likelihood_spam[i,vocab_absent]))+log(prior_spam) # log of P(D|w=S)*P(w=S)
            lxplog_ham <- sum(log(likelihood_ham[i,vocab_present]))+sum(log(1-likelihood_ham[i,vocab_absent]))+log(prior_ham) # log of P(D|w=H)*P(w=H)
            # note: actual probabilities:
            #     P(w=S|D) is exp(lxplog_spam)/(exp(lxplog_spam)+exp(lxplog_ham))
            #     P(w=H|D) is exp(lxplog_ham)/(exp(lxplog_spam)+exp(lxplog_ham))
            # however, due to the very small values (approaching zero) which will cause loss of accuracy on the results, 
            # I decided to simplify the equation, divide out the denominators, and just compare the exponent of the numerators: 
            # since: exp(-n) > (<) exp(-m) is equal to -n > (<) -m
            probability_spam <- lxplog_spam
            probability_ham <- lxplog_ham

            if(is.na(probability_spam) || is.na(probability_ham)){
                classification[i,j] <- "ham" # if can't classify, assume as ham
            }else if(probability_spam > probability_ham){
                classification[i,j] <- "spam"
            }else if(probability_spam < probability_ham){
                classification[i,j] <- "ham"
            }else{
                classification[i,j] <- "ham" # if can't classify, assume as ham
            }

            iterate_precision_recall(dataset$type[idx], classification[i,j], i)
        }
    }


    # dump to files the following:
    #     filename, true classification, computed classification
    #     report precision and recall for each lambdas
    for(i in 1:L){
        write(paste(dataset$doc[testset_idx], dataset$type[testset_idx], classification[i,], sep=" "), outputfile[i]) # write to a file, space delimited
        cat(paste0("[Lambda ", lambdalist[i], "]\tHam/Spam predictions are saved to ", outputfile[i], "."), sep="\n")

        precision[i] <<- compute_precision(i)
        recall[i] <<- compute_recall(i)
        cat("\n\n", file=outputfile[i], fill=TRUE, append=TRUE)
        cat(paste(precision[i], recall[i]), file=outputfile[i], fill=TRUE, append=TRUE)
    }
}

<h2>Part 2: Main Program</h2>

<h4>Instructions for user:</h4>
<ul>
    <li>Specify in <b>default_dir</b> the location of your code and files.</li>
    <li>Specify in <b>data_dir</b> the location of your trec06p dataset. Note, this is only tested on trec06p dataset and using a different corpus with different formatting might need parsing changes.</li>
    <li>Specify in <b>working_dir</b> the directory location of labels file, and where the output files of this program should be saved.</li>
</ul>

In [8]:
default_dir <- '/'
data_dir <- paste(default_dir, "trec06p", sep="/")
working_dir <- paste(default_dir, "results", sep="/")
setwd(working_dir)

setup_dataset("labels")

[1] "Count of all documents:  37822"
[1] "Count of training set:  26476"
[1] "Count of test set:  11346"


<h4> Instructions for user: </h4>

<b>MENU 1:</b> This program can be run with different settings. Change the value of <b>choice1</b> based on your run preference.
<ol>
    <li>No Lambda Smoothing</li>
    <li>With Lambda Smoothing (run on all lambdas)</li>
    <li>200 Most Informative Words</li>
</ol>

In [9]:
choice1 <- 2  # change this value based on the run setting preferred

if (choice1 == 1) {
    print("Selected: [1] No Lambda Smoothing")
    lambdalist <- c(0.000)
    vocabfile <- "vocabulary.txt"
    outfile <- "result"
    normal <- TRUE
} else if (choice1 == 2) {
    print("Selected: [2] With Lambda Smoothing (run on all lambdas)")
    lambdalist <- c(0.005, 0.100, 0.500, 1.000, 2.000)
    vocabfile <- "vocabulary.txt"
    outfile <- "result"
    normal <- TRUE
} else if (choice1 == 3) {
    print("Selected: [3] 200 Most Informative Words")
    lambdalist <- c(0.500)
    vocabfile <- "vocabulary.200.txt"
    outfile <- "result_200"
    normal <- FALSE
}

[1] "Selected: [2] With Lambda Smoothing (run on all lambdas)"


<h4> Instructions for user: </h4>

<b>MENU 2:</b> This program can be run with different settings. Change the value of <b>choice2</b> based on your run preference.
<ol>
    <li>Construct new vocabulary</li>
    <li>Load vocabulary from file (requires existing vocabulary.txt)</li>
</ol>

In [10]:
choice2 <- 2 # change this value based on the run setting preferred

if (choice2 == 1) {
    print("Selected: [1] Construct new vocabulary")
    print("Constructing vocabulary from training set.")
    print("This might take a while...")
    construct_vocabulary(vocabfile,normal)
    print(date())
} else {
    print("Selected: [2] Load vocabulary from file")
    print("Loading vocabulary from file...")
    load_vocabulary(vocabfile)
    print(date())
}

[1] "Selected: [2] Load vocabulary from file"
[1] "Loading vocabulary from file..."
[1] "Sat Sep 18 15:09:18 2021"


In [11]:
print("Computing priors and likelihoods...")
compute_priors()
# compute likelihoods for all lambdas, computed simultaneously for faster processing time
# initializing
likelihood_spam <- matrix(data=0, nrow=length(lambdalist), ncol=length(vocabulary), byrow=FALSE, dimnames=list(c(1:length(lambdalist)), vocabulary))
likelihood_ham <- matrix(data=0, nrow=length(lambdalist), ncol=length(vocabulary), byrow=FALSE, dimnames=list(c(1:length(lambdalist)), vocabulary))
for(i in 1:length(lambdalist))
    compute_likelihoods(i)

[1] "Computing priors and likelihoods..."


In [12]:
print("Classifying documents on test set.")
print("This might take a while...")
run_classifier(outfile, normal)
# print precision recall in separate file
cat(lambdalist, file="precision_recall_report.txt", fill=TRUE, append=FALSE)
cat(precision, file="precision_recall_report.txt", fill=TRUE, append=TRUE)
cat(recall, file="precision_recall_report.txt", fill=TRUE, append=TRUE)
cat("\n\nPrecision and recall of the test set per lambda.\n")
cat("\n[L]\t[P]\t[R]\n")
cat(paste(lambdalist, round(precision, digits=4), round(recall, digits=4), sep="\t"), sep="\n")
cat("\nIt is also saved in precision_recall_report.txt.\n")
cat(date())

[1] "Classifying documents on test set."
[1] "This might take a while..."


Parsing test set document # 1
Parsing test set document # 501
Parsing test set document # 1001
Parsing test set document # 1501
Parsing test set document # 2001
Parsing test set document # 2501
Parsing test set document # 3001
Parsing test set document # 3501
Parsing test set document # 4001
Parsing test set document # 4501
Parsing test set document # 5001
Parsing test set document # 5501
Parsing test set document # 6001
Parsing test set document # 6501
Parsing test set document # 7001
Warning message in scan(file = document, "", quote = ""):
“embedded nul(s) found in input”Parsing test set document # 7501
Parsing test set document # 8001
Parsing test set document # 8501
Parsing test set document # 9001
Parsing test set document # 9501
Parsing test set document # 10001
Parsing test set document # 10501
Parsing test set document # 11001
Parsing test set document # 11346


[Lambda 0.005]	Ham/Spam predictions are saved to result.0.005.txt.
[Lambda 0.1]	Ham/Spam predictions are saved to result.0.100.txt.
[Lambda 0.5]	Ham/Spam predictions are saved to result.0.500.txt.
[Lambda 1]	Ham/Spam predictions are saved to result.1.000.txt.
[Lambda 2]	Ham/Spam predictions are saved to result.2.000.txt.


Precision and recall of the test set per lambda.

[L]	[P]	[R]
0.005	0.9603	0.983
0.1	0.9826	0.9785
0.5	0.9937	0.9821
1	0.9931	0.984
2	0.9915	0.9863

It is also saved in precision_recall_report.txt.
Sat Sep 18 15:31:33 2021